# ML Models

## Import Libraries & Load Dataframe from AWS DB

In [250]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from math import sin, cos, sqrt, atan2, radians

import query_helper
import get_new_route
import json


## Get input from user for recommendation

In [271]:
df_numeric = pd.read_csv('df_numeric.csv', index_col='id')
df_numeric.head()

,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
105714722,Central Yellow Wall,V3 R,4.4,22,0,South Dakota,Custer State Park,Sylvan Lake,Sylvan Lake Bouldering,Campground Boulder,-103.564,43.8463,https://www.mountainproject.com/route/10571472...,0,0,1,1,0,0,0,0,0,0,0,0,The Central Yellow Wall is a classic must-do p...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
105714728,Waves,5.8,4.6,217,2,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Shipyard Rock,-103.466,43.8865,https://www.mountainproject.com/route/10571472...,1,0,0,0,0,0,0,0,0,0,14,0,Waves is one of the best easy routes at Rushmo...,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
105714731,East Chimney Variation,5.7,4.1,40,2,South Dakota,Custer State Park,Cathedral Spires,Station 13,0,-103.534,43.8488,https://www.mountainproject.com/route/10571473...,0,1,0,0,0,0,0,0,0,0,11,0,Start at the base of the easy chimney on the e...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
105714734,Conn Diagonal,5.7,4.9,151,3,South Dakota,Custer State Park,Sylvan Lake,Outlets,Outer Outlet,-103.568,43.8487,https://www.mountainproject.com/route/10571473...,0,1,0,0,0,0,0,0,0,0,11,0,This is one of the most classic 5.7 routes tha...,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
105714737,Bolts for Bobs,5.8,3.5,125,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Borneo,-103.469,43.8870,https://www.mountainproject.com/route/10571473...,1,0,0,0,0,0,0,0,0,0,14,0,The rightmost line on the rock (see directions...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [272]:
target_id = 109341545
target_lat = 32.9127 
target_lon = -116.882
target_state =None
target_city =None
target_zipcode = '10032'
target_radius_range=None
star_limit = 0
###other parameters to be added here later

### Get coordinates for zip or city

In [273]:
with open('us-zip-code-latitude-and-longitude.json') as f:
  coord_dict = json.load(f)

In [274]:
def get_coords(city=None, state=None, zipcode=None):
    #find the coordinates for city or zip code
    for city in coord_dict:
        if city['fields']['zip']==zipcode:
            return city['fields']['latitude'],city['fields']['longitude']
        if (city['fields']['state']==state)&(city['fields']['city']==city):
            return city['fields']['latitude'],city['fields']['longitude']
    #if nothing is found return none
    return None, None

In [275]:
target_lat, target_lon = get_coords(target_city, target_state, target_zipcode)

In [276]:
print(target_lat, target_lon)

40.840686 -73.94154


### To begin, see if if the climb already exists in db

In [278]:
if target_id in df_numeric.index:
    print('We have climb already')
else:
    print('Making API call and Scraping climb data')
    if(get_new_route.get_route_details(target_id)):
        df_target= pd.read_csv('target_climb.csv', index_col= 'id')
        df_target.drop(columns=['Unnamed: 0'], inplace=True)
        df_target['in_range'] = 1
        df_target = df_target[['name', 'rating', 'stars', 'starVotes', 'pitches', 'location', 'region',
                               'area', 'sub_area', 'wall', 'longitude', 'latitude', 'url', 'Sport',
                               'Trad', 'Boulder', 'TR', 'Alpine', 'Aid', 'Ice', 'Snow', 'Mixed',
                               'danger', 'rope_grade', 'boulder_grade', 'infos', 'slab', 'traverse',
                               'roof', 'corner', 'crack', 'face', 'flake', 'fingers', 'jug', 'exposed',
                               'dihedral', 'sustained', 'technical', 'run out', 'well protected',
                               'chimney', 'offwidth', 'stem', 'arete', 'crimp', 'vertical', 'powerful',
                               'in_range']]
        
        df_numeric = pd.concat([df_numeric, df_target])
        df_numeric.tail()
    else:
        print("Something went wrong")


We have climb already


## Reccomender

#### Kernel Imports

In [213]:
# Import kernels
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import polynomial_kernel
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import laplacian_kernel
from sklearn.metrics.pairwise import chi2_kernel



In [214]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler

#### Create fxn to see if climb is in search range

In [215]:
#function takes search param range and assigns to original df if climb in_range
def in_range(df_fxn, lat, lon, radius_range=None):
    if radius_range:
        R= 3958.8 
        if (lat == None)|(lon==None):
            df_fxn['in_range'] = 1
        else:
            #assign target coords and set to radians for calc
            lat1 = radians(lat)
            lon1 = radians(lon)
            for index, row in df_fxn.iterrows():
                #assign the lat and lon for each climb
                lat2 = radians(row['latitude'])
                lon2 = radians(row['longitude'])

                dlon = lon2 - lon1
                dlat = lat2 - lat1

                a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
                c = 2 * atan2(sqrt(a), sqrt(1 - a))

                distance = R * c

                #assign in_range col to 1 if the climb is in range
                if distance < radius_range:
                    df_fxn.at[index,'in_range']=1
                else:
                    df_fxn.at[index,'in_range']=0   
    else:
        df_fxn['in_range'] =1



In [216]:
def star_cutoff(df_fxn, star_limit=3.5):
    for index, row in df_fxn.iterrows():
        #assign in_range col to 1 if the climb is in range
        if df_fxn.at[index, 'stars'] >= star_limit:
            df_fxn.at[index,'in_range']=1
        else:
            df_fxn.at[index,'in_range']=0   

#### Call function to assign if climb in range

In [217]:
## used to get list of climbs allowed for comparison
in_range(df_numeric, lat = target_lat, lon = target_lon, radius_range=target_radius_range)

#### Star cutoff (ie only give results for routes with above 3.5 stars)

In [218]:
star_cutoff(df_numeric, star_limit)

#### Create df_in_range to run recommender in subset

In [232]:
df_in_range = df_numeric[df_numeric['in_range']==1].reset_index()      #maybe add star filter in this line
target_index =df_in_range.index[df_in_range['id']==target_id][0]
df_in_range.shape

(36609, 50)

### Scale Features

#### Create Features DF

In [233]:
#creates features df used for comparison
features = df_in_range.loc[:,['stars', 'pitches', 'Sport', 'Trad', 'Boulder', 'TR', 'Alpine', 'Aid',
       'Ice', 'Snow', 'Mixed', 'danger', 'rope_grade', 'boulder_grade', 'slab', 'traverse', 'roof', 
                'corner', 'crack', 'face','flake', 'fingers',
                 'jug', 'exposed', 'dihedral', 'sustained', 'technical','run out', 'well protected',
                 'chimney', 'offwidth', 'stem', 'arete','crimp', 'vertical', 'powerful']] #,'longitude','latitude',

In [234]:
features.shape

(36609, 36)

In [235]:
# features.iloc[:,13].value_counts()

#### Pick scaling type (AND UPDATE WEIGHTS)

In [236]:
min_max_scaler = MinMaxScaler()
scalar = StandardScaler()

In [237]:
##### Pick a scaling option ###############################

# features_scaled = scalar.fit_transform(features)
features_scaled = min_max_scaler.fit_transform(features.drop(columns=['danger','pitches']))

# scale danger and pitches using ss and add into features scaled df
features_scaled = np.concatenate((features_scaled, scalar.fit_transform(features[['danger', 'pitches']])), axis=1)

##################################################################

#### Add weight for difficulity of climb

In [238]:
for i in range(features_scaled.shape[0]):
    features_scaled[i][10]=features_scaled[i][10]*5
    features_scaled[i][11]=features_scaled[i][11]*5

### Now lets fit the similarity model

#### Rec function

In [239]:
def get_recommendations(idx, kernel_type):

    #value to store scores and indicies
    score_matrix = np.ndarray(shape=(len(df_in_range),2), dtype=float)

    #go through the target climb vs all onthers in our db and populate score mtx with index and similarity
    for i in range(df_in_range.shape[0]):
        score = kernel_type(features_scaled[idx].reshape(1,-1),features_scaled[i].reshape(1,-1))
        score_matrix[i][0] =  i        ##the index comparison corresponding to the score
        score_matrix[i][1] = score     ##the score for the current index

    # Sort the climbs based on the similarity scores
    score_matrix = sorted(score_matrix, key=lambda x: x[1], reverse=True)
    
    # score_matrix
#########################  ADD/calculate SIMilarity VALUE   ###########################

    # # Get the scores of the 20 most similar climbs
    score_matrix = score_matrix[1:20]

    # # Get the climb indices (& cast to ints)
    climb_indices = [int(i[0]) for i in score_matrix]
    
    # Return the top 20 most similar climbs
    return df_in_range.loc[climb_indices,:]

#### Cells for comparison (delete later)

In [240]:
df_numeric.loc[105721615, :]

name                                                   Double Cross
rating                                                         5.7+
stars                                                           4.5
starVotes                                                      1271
pitches                                                           1
location                                                 California
region                                    Joshua Tree National Park
area                                       Hidden Valley Campground
sub_area                                              The Old Woman
wall                                      The Old Woman - West Face
longitude                                                  -116.164
latitude                                                    34.0165
url               https://www.mountainproject.com/route/10572161...
Sport                                                             0
Trad                                            

In [241]:
df_numeric.shape

(36609, 49)

In [242]:
target_index

304

In [243]:
# df_numeric[(df_numeric['boulder_grade']>5)]

#### Call rec fxn

In [244]:
pd.options.display.max_columns = None

In [245]:
rec=get_recommendations(target_index, cosine_similarity)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
22904,109498932,Bon Temps Rouler,5.9,4.2,27,1,New Hampshire,WM: Kancamagus (Eastern),Sundown Ledge,Main Cliff,0,-71.2393,44.0050,https://www.mountainproject.com/route/10949893...,0,1,0,0,0,0,0,0,0,0,17,0,A really nice moderate crack climb with a lot ...,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
13373,106862754,Tribal Therapy,5.8+,5.0,5,1,Washington,Southwest Cascades,Farside / Dropzone,(2) Dropzone Area,0,-122.2060,45.5691,https://www.mountainproject.com/route/10686275...,0,1,0,0,0,0,0,0,0,0,14,0,Great route! Scramble up the broken pillar sec...,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
27594,111987909,Cracksters Unite,5.8,4.0,1,1,Montana,Southwest Region,Homestake Pass,Split Pinnacle,0,-112.3710,45.9170,https://www.mountainproject.com/route/11198790...,0,1,0,0,0,0,0,0,0,0,14,0,"Thin hands crack followed by a traverse, then ...",0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
6934,105996559,Penelope's Problem,5.7,3.4,200,1,California,Yosemite National Park,Yosemite Valley,Valley North Side,"Yosemite Falls Area,Swan Slab",-119.6000,37.7451,https://www.mountainproject.com/route/10599655...,0,1,0,0,0,0,0,0,0,0,11,0,"Just left of Swan Slab Gully is this fun, shor...",0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
311,105721639,Geronimo,5.7,4.0,127,1,California,Joshua Tree National Park,Hidden Valley Campground,The Old Woman,The Old Woman - East Face,-116.1640,34.0167,https://www.mountainproject.com/route/10572163...,0,1,0,0,0,0,0,0,0,0,11,0,I'm sure there are many ways to get to this ro...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
23314,109651586,Never Alone,5.7,4.0,1,1,New York,Adirondacks,G: Indian Lake Region,Crane Mountain,"Sky High,Sky High Wall",-73.9565,43.5406,https://www.mountainproject.com/route/10965158...,0,1,0,0,0,0,0,0,0,0,11,0,"Interesting FA story: my rope got jammed, and ...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
33815,116598226,Lost Hands,5.7+,4.0,1,1,California,Central Coast,Santa Barbara,The Playground,Land of the Lost,-119.8460,34.5010,https://www.mountainproject.com/route/11659822...,0,1,0,0,0,0,0,0,0,0,11,0,"As my brother might say, the crux is deciding ...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1384,105741374,Fear of Gear,5.7,3.8,53,1,Utah,Uinta Mountains,Stone Garden,0,0,-110.8790,40.7498,https://www.mountainproject.com/route/10574137...,0,1,0,0,0,0,0,0,0,0,11,0,This climb is located on the far left of side ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21924,108936684,Putrid Rat,5.7,3.8,12,1,California,Central Coast,Santa Barbara,Panic Town,Wall 3,-119.7060,34.4786,https://www.mountainproject.com/route/10893668...,0,1,0,0,0,0,0,0,0,0,11,0,The long hand crack directly underneath the ra...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21820,108876736,Another Quick Shower,5.7+,3.7,6,1,California,Central Coast,Santa Barbara,Panic Town,Wall 1,-119.7060,34.4786,https://www.mountainproject.com/route/10887673...,0,1,0,0,0,0,0,0,0,0,11,0,"The thin-looking, ragged crack between Lovely ...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [247]:
rec=get_recommendations(target_index, rbf_kernel)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
311,105721639,Geronimo,5.7,4.0,127,1,California,Joshua Tree National Park,Hidden Valley Campground,The Old Woman,The Old Woman - East Face,-116.1640,34.0167,https://www.mountainproject.com/route/10572163...,0,1,0,0,0,0,0,0,0,0,11,0,I'm sure there are many ways to get to this ro...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
23314,109651586,Never Alone,5.7,4.0,1,1,New York,Adirondacks,G: Indian Lake Region,Crane Mountain,"Sky High,Sky High Wall",-73.9565,43.5406,https://www.mountainproject.com/route/10965158...,0,1,0,0,0,0,0,0,0,0,11,0,"Interesting FA story: my rope got jammed, and ...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
33815,116598226,Lost Hands,5.7+,4.0,1,1,California,Central Coast,Santa Barbara,The Playground,Land of the Lost,-119.8460,34.5010,https://www.mountainproject.com/route/11659822...,0,1,0,0,0,0,0,0,0,0,11,0,"As my brother might say, the crux is deciding ...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1384,105741374,Fear of Gear,5.7,3.8,53,1,Utah,Uinta Mountains,Stone Garden,0,0,-110.8790,40.7498,https://www.mountainproject.com/route/10574137...,0,1,0,0,0,0,0,0,0,0,11,0,This climb is located on the far left of side ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21924,108936684,Putrid Rat,5.7,3.8,12,1,California,Central Coast,Santa Barbara,Panic Town,Wall 3,-119.7060,34.4786,https://www.mountainproject.com/route/10893668...,0,1,0,0,0,0,0,0,0,0,11,0,The long hand crack directly underneath the ra...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21820,108876736,Another Quick Shower,5.7+,3.7,6,1,California,Central Coast,Santa Barbara,Panic Town,Wall 1,-119.7060,34.4786,https://www.mountainproject.com/route/10887673...,0,1,0,0,0,0,0,0,0,0,11,0,"The thin-looking, ragged crack between Lovely ...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6456,105963178,St. Alfonso's,5.7,3.6,45,1,Kentucky,Red River Gorge,Phantasia,0,0,-83.6428,37.8431,https://www.mountainproject.com/route/10596317...,0,1,0,0,0,0,0,0,0,0,11,0,A fun route worth climbing. Chimney up to gain...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
312,105721645,Buissonier,5.7+,3.5,291,1,California,Joshua Tree National Park,Hidden Valley Campground,The Blob,The Blob - Southeast Face,-116.1640,34.0179,https://www.mountainproject.com/route/10572164...,0,1,0,0,0,0,0,0,0,0,11,0,I never have figured out quite how to pronounc...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
425,105722221,Leaping Leaner,5.7,3.5,246,1,California,Joshua Tree National Park,Real Hidden Valley,Locomotion Rock,0,-116.1680,34.0134,https://www.mountainproject.com/route/10572222...,0,1,0,0,0,0,0,0,0,0,11,0,Follow the directions to reach the Locomotion ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21660,108779464,Top of the Pops.,5.7,3.5,18,1,California,San Diego and Environs,South San Diego County,Mission Gorge,Limbo Area,-117.0500,32.8262,https://www.mountainproject.com/route/10877946...,0,1,0,0,0,0,0,0,0,0,11,0,One of the best splitters at The Gorge. Arguab...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [248]:
rec=get_recommendations(target_index, laplacian_kernel)
rec

,id,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
311,105721639,Geronimo,5.7,4.0,127,1,California,Joshua Tree National Park,Hidden Valley Campground,The Old Woman,The Old Woman - East Face,-116.1640,34.0167,https://www.mountainproject.com/route/10572163...,0,1,0,0,0,0,0,0,0,0,11,0,I'm sure there are many ways to get to this ro...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
23314,109651586,Never Alone,5.7,4.0,1,1,New York,Adirondacks,G: Indian Lake Region,Crane Mountain,"Sky High,Sky High Wall",-73.9565,43.5406,https://www.mountainproject.com/route/10965158...,0,1,0,0,0,0,0,0,0,0,11,0,"Interesting FA story: my rope got jammed, and ...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
33815,116598226,Lost Hands,5.7+,4.0,1,1,California,Central Coast,Santa Barbara,The Playground,Land of the Lost,-119.8460,34.5010,https://www.mountainproject.com/route/11659822...,0,1,0,0,0,0,0,0,0,0,11,0,"As my brother might say, the crux is deciding ...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1384,105741374,Fear of Gear,5.7,3.8,53,1,Utah,Uinta Mountains,Stone Garden,0,0,-110.8790,40.7498,https://www.mountainproject.com/route/10574137...,0,1,0,0,0,0,0,0,0,0,11,0,This climb is located on the far left of side ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21924,108936684,Putrid Rat,5.7,3.8,12,1,California,Central Coast,Santa Barbara,Panic Town,Wall 3,-119.7060,34.4786,https://www.mountainproject.com/route/10893668...,0,1,0,0,0,0,0,0,0,0,11,0,The long hand crack directly underneath the ra...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21820,108876736,Another Quick Shower,5.7+,3.7,6,1,California,Central Coast,Santa Barbara,Panic Town,Wall 1,-119.7060,34.4786,https://www.mountainproject.com/route/10887673...,0,1,0,0,0,0,0,0,0,0,11,0,"The thin-looking, ragged crack between Lovely ...",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6456,105963178,St. Alfonso's,5.7,3.6,45,1,Kentucky,Red River Gorge,Phantasia,0,0,-83.6428,37.8431,https://www.mountainproject.com/route/10596317...,0,1,0,0,0,0,0,0,0,0,11,0,A fun route worth climbing. Chimney up to gain...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
312,105721645,Buissonier,5.7+,3.5,291,1,California,Joshua Tree National Park,Hidden Valley Campground,The Blob,The Blob - Southeast Face,-116.1640,34.0179,https://www.mountainproject.com/route/10572164...,0,1,0,0,0,0,0,0,0,0,11,0,I never have figured out quite how to pronounc...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
425,105722221,Leaping Leaner,5.7,3.5,246,1,California,Joshua Tree National Park,Real Hidden Valley,Locomotion Rock,0,-116.1680,34.0134,https://www.mountainproject.com/route/10572222...,0,1,0,0,0,0,0,0,0,0,11,0,Follow the directions to reach the Locomotion ...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21660,108779464,Top of the Pops.,5.7,3.5,18,1,California,San Diego and Environs,South San Diego County,Mission Gorge,Limbo Area,-117.0500,32.8262,https://www.mountainproject.com/route/10877946...,0,1,0,0,0,0,0,0,0,0,11,0,One of the best splitters at The Gorge. Arguab...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [102]:
rec=get_recommendations(target_index, euclidean_distances)
rec

,id,orig_index,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
25580,111057266,25580,Laces Out aka Houdini Traverse,V9,0.0,0,0,California,Redwood Coast,Houda Point,Houda North,0,-124.1190,41.0359,https://www.mountainproject.com/route/11105726...,0,0,1,0,0,0,0,0,0,0,0,39,from start of houda crack traverse right to th...,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
17568,107648102,17568,Yabo Squeeze,V9,3.0,1,0,California,San Francisco Bay Area,Castle Rock Area,* Castle Rock Area Bouldering,"Castle Rock Loop,Parking Lot Rock",-122.0950,37.2296,https://www.mountainproject.com/route/10764810...,0,0,1,0,0,0,0,0,0,0,0,39,This is a squeeze job and quasi eliminate betw...,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
27001,111798944,27001,Breaker of Chains,V9,3.0,1,0,Massachusetts,Charlemont,Upper Field,0,0,-72.7751,42.7223,https://www.mountainproject.com/route/11179894...,0,0,1,0,0,0,0,0,0,0,0,39,Start standing with a right hand on a sloping ...,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
17323,107591323,17323,The Graduate,V9,4.0,1,0,Colorado,Gunnison,Taylor Canyon,Taylor Canyon Bouldering,Esty Lane Boulders,-106.7420,38.7327,https://www.mountainproject.com/route/10759132...,0,0,1,0,0,0,0,0,0,0,0,39,Stand start in the rubble on a huge undercling...,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3125,105761781,3125,One Ton Ho,V9,4.2,5,0,Colorado,Fort Collins,Poudre Canyon,West of Rustic,"420s,Hank's Boulder",-105.8060,40.6731,https://www.mountainproject.com/route/10576178...,0,0,1,0,0,0,0,0,0,0,0,39,This is located on the Hank's Boulder. It the...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
11348,106566855,11348,Bierstadt,V9+,4.9,18,0,Colorado,Alpine Rock,Mt. Evans,Mt. Evans Bouldering,"Area A,Bierstadt Area",-105.6210,39.6326,https://www.mountainproject.com/route/10656685...,0,0,1,0,1,0,0,0,0,0,0,39,"Originally considered V10, the variety of diff...",0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1
10673,106476136,10673,Deep Puddle Dynamics,V9+,4.8,12,0,Colorado,Alpine Rock,RMNP - Rock,RMNP Bouldering Areas,"Chaos Canyon Bouldering,Lower Chaos Canyon,Dee...",-105.6610,40.3048,https://www.mountainproject.com/route/10647613...,0,0,1,0,1,0,0,0,0,0,0,39,Perfect slopers on a gently overhanging face. ...,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21238,108561805,21238,Linder's Roof,V9,4.4,11,0,Utah,Saint George,Moe's Valley,Tee Pee Area,Ripple,-113.6300,37.0655,https://www.mountainproject.com/route/10856180...,0,0,1,0,0,0,0,0,0,0,0,39,Hard moves right off the ground to an amazing ...,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
27203,111866060,27203,The Carnivore Stand,V9,2.0,1,0,Massachusetts,Charlemont,Upper Field,0,0,-72.7751,42.7223,https://www.mountainproject.com/route/11186606...,0,0,1,0,0,0,0,0,0,0,0,39,Start with your right hand on a full pad under...,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15647,107273813,15647,Red Dwarf,V9,2.5,2,0,New Mexico,Socorro Area,The Box Climbing Areas,Spook Canyon,Spook Outlying Boulders,-106.9820,34.0002,https://www.mountainproject.com/route/10727381...,0,0,1,0,0,0,0,0,0,0,0,39,Starting low on a right hand slot and left han...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [103]:
rec=get_recommendations(target_index, linear_kernel)
rec

,id,orig_index,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
2060,105750802,2060,Fool's Gold,5.9+,4.5,17,2,Colorado,South Platte,Goose Creek/Molly Gulch Campground,Helen's Dome,0,-105.3350,39.1887,https://www.mountainproject.com/route/10575080...,1,1,0,0,0,0,0,0,0,0,17,0,Start off the ramp about 30 feet left of the l...,1,0,0,0,1,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1
7452,106049635,7452,W.A.S.P. (center variation start),5.9+,4.4,44,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Hornets Nest,-103.4670,43.8864,https://www.mountainproject.com/route/10604963...,1,1,0,0,0,0,0,0,0,0,17,0,This is a nice solid 5.9 which is consistent a...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
21918,108935327,21918,Captain Hook,5.9 PG13,4.4,7,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Sharks Fin,-103.4670,43.8863,https://www.mountainproject.com/route/10893532...,1,1,0,0,0,0,0,0,0,1,17,0,Starts on left to right angling dike. Clip bol...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1167,105738359,1167,Fort Stress,5.9+,4.3,52,4,Arizona,Southern Arizona,Mount Lemmon (Santa Catalina Mountains),Mount Lemmon (Catalina Highway),"8 - Summit Crags,The Fortress,Southeast Face",-110.7980,32.4321,https://www.mountainproject.com/route/10573835...,1,0,0,0,0,0,0,0,0,0,17,0,"This route is long, fun, reasonably safe, high...",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
22675,109364756,22675,D4,5.9+,4.2,62,6,Colorado,Buena Vista,Davis Face,0,0,-106.1050,38.9174,https://www.mountainproject.com/route/10936475...,1,0,0,0,0,0,0,0,0,0,17,0,D4 is a beautiful route on the right side of t...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
31316,114262472,31316,Void of Form,5.9,4.0,47,10,Arizona,Southern Arizona,Mount Lemmon (Santa Catalina Mountains),Pontatoc Canyon,Pontatoc Ridge,-110.8880,32.3454,https://www.mountainproject.com/route/11426247...,1,0,0,0,0,0,0,0,0,0,17,0,Void of Form climbs the long buttress of rock ...,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1
25125,110794133,25125,Guava Jelly,5.9+,4.1,29,1,Utah,Moab Area,La Sal Mountains,Upper Dakota Crag,Trenchtown,-109.2740,38.4703,https://www.mountainproject.com/route/11079413...,1,0,0,0,0,0,0,0,0,0,17,0,An unbelievable dihedral climb with lots of sm...,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
6358,105956634,6358,Camel Toe,5.9+,4.0,38,6,Colorado,Estes Park Valley,Fall River Road (US 34 West),McGregor Slab,0,-105.5850,40.4085,https://www.mountainproject.com/route/10595663...,1,1,0,0,0,0,0,0,0,0,17,0,This is a writeup is based upon firsthand expe...,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7289,106028627,7289,Rock Du Jours Direct,5.9+,4.0,179,2,New Hampshire,*Rumney,Main Cliff,0,0,-71.8359,43.8025,https://www.mountainproject.com/route/10602862...,1,0,0,0,0,0,0,0,0,0,17,0,This is perhaps one of the best 5.9s at Rumney...,1,1,0,1,1,1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
1097,105737426,1097,Thin Man,5.9 R,4.0,29,1,California,Central Coast,San Luis Obispo,Bishop Peak,Shadow Rock,-120.6960,35.3037,https://www.mountainproject.com/route/10573742...,1,0,0,0,0,0,0,0,0,2,17,0,The last route to the left of Shadow before yo...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [107]:
rec=get_recommendations(target_index, polynomial_kernel)
rec

,id,orig_index,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
792,105750802,2060,Fool's Gold,5.9+,4.5,17,2,Colorado,South Platte,Goose Creek/Molly Gulch Campground,Helen's Dome,0,-105.3350,39.1887,https://www.mountainproject.com/route/10575080...,1,1,0,0,0,0,0,0,0,0,17,0,Start off the ramp about 30 feet left of the l...,1,0,0,0,1,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1
3066,106049635,7452,W.A.S.P. (center variation start),5.9+,4.4,44,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Hornets Nest,-103.4670,43.8864,https://www.mountainproject.com/route/10604963...,1,1,0,0,0,0,0,0,0,0,17,0,This is a nice solid 5.9 which is consistent a...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9047,108935327,21918,Captain Hook,5.9 PG13,4.4,7,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Sharks Fin,-103.4670,43.8863,https://www.mountainproject.com/route/10893532...,1,1,0,0,0,0,0,0,0,1,17,0,Starts on left to right angling dike. Clip bol...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
441,105738359,1167,Fort Stress,5.9+,4.3,52,4,Arizona,Southern Arizona,Mount Lemmon (Santa Catalina Mountains),Mount Lemmon (Catalina Highway),"8 - Summit Crags,The Fortress,Southeast Face",-110.7980,32.4321,https://www.mountainproject.com/route/10573835...,1,0,0,0,0,0,0,0,0,0,17,0,"This route is long, fun, reasonably safe, high...",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9382,109364756,22675,D4,5.9+,4.2,62,6,Colorado,Buena Vista,Davis Face,0,0,-106.1050,38.9174,https://www.mountainproject.com/route/10936475...,1,0,0,0,0,0,0,0,0,0,17,0,D4 is a beautiful route on the right side of t...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
10565,110794133,25125,Guava Jelly,5.9+,4.1,29,1,Utah,Moab Area,La Sal Mountains,Upper Dakota Crag,Trenchtown,-109.2740,38.4703,https://www.mountainproject.com/route/11079413...,1,0,0,0,0,0,0,0,0,0,17,0,An unbelievable dihedral climb with lots of sm...,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
13591,114262472,31316,Void of Form,5.9,4.0,47,10,Arizona,Southern Arizona,Mount Lemmon (Santa Catalina Mountains),Pontatoc Canyon,Pontatoc Ridge,-110.8880,32.3454,https://www.mountainproject.com/route/11426247...,1,0,0,0,0,0,0,0,0,0,17,0,Void of Form climbs the long buttress of rock ...,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1
2676,105956634,6358,Camel Toe,5.9+,4.0,38,6,Colorado,Estes Park Valley,Fall River Road (US 34 West),McGregor Slab,0,-105.5850,40.4085,https://www.mountainproject.com/route/10595663...,1,1,0,0,0,0,0,0,0,0,17,0,This is a writeup is based upon firsthand expe...,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2999,106028627,7289,Rock Du Jours Direct,5.9+,4.0,179,2,New Hampshire,*Rumney,Main Cliff,0,0,-71.8359,43.8025,https://www.mountainproject.com/route/10602862...,1,0,0,0,0,0,0,0,0,0,17,0,This is perhaps one of the best 5.9s at Rumney...,1,1,0,1,1,1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
429,105737426,1097,Thin Man,5.9 R,4.0,29,1,California,Central Coast,San Luis Obispo,Bishop Peak,Shadow Rock,-120.6960,35.3037,https://www.mountainproject.com/route/10573742...,1,0,0,0,0,0,0,0,0,2,17,0,The last route to the left of Shadow before yo...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [108]:
rec=get_recommendations(target_index, sigmoid_kernel)
rec

,id,orig_index,name,rating,stars,starVotes,pitches,location,region,area,sub_area,wall,longitude,latitude,url,Sport,Trad,Boulder,TR,Alpine,Aid,Ice,Snow,Mixed,danger,rope_grade,boulder_grade,infos,slab,traverse,roof,corner,crack,face,flake,fingers,jug,exposed,dihedral,sustained,technical,run out,well protected,chimney,offwidth,stem,arete,crimp,vertical,powerful,in_range
792,105750802,2060,Fool's Gold,5.9+,4.5,17,2,Colorado,South Platte,Goose Creek/Molly Gulch Campground,Helen's Dome,0,-105.3350,39.1887,https://www.mountainproject.com/route/10575080...,1,1,0,0,0,0,0,0,0,0,17,0,Start off the ramp about 30 feet left of the l...,1,0,0,0,1,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1
3066,106049635,7452,W.A.S.P. (center variation start),5.9+,4.4,44,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Hornets Nest,-103.4670,43.8864,https://www.mountainproject.com/route/10604963...,1,1,0,0,0,0,0,0,0,0,17,0,This is a nice solid 5.9 which is consistent a...,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9047,108935327,21918,Captain Hook,5.9 PG13,4.4,7,1,South Dakota,The Needles Of Rushmore,Mount Rushmore National Memorial,South Seas,Sharks Fin,-103.4670,43.8863,https://www.mountainproject.com/route/10893532...,1,1,0,0,0,0,0,0,0,1,17,0,Starts on left to right angling dike. Clip bol...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
441,105738359,1167,Fort Stress,5.9+,4.3,52,4,Arizona,Southern Arizona,Mount Lemmon (Santa Catalina Mountains),Mount Lemmon (Catalina Highway),"8 - Summit Crags,The Fortress,Southeast Face",-110.7980,32.4321,https://www.mountainproject.com/route/10573835...,1,0,0,0,0,0,0,0,0,0,17,0,"This route is long, fun, reasonably safe, high...",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9382,109364756,22675,D4,5.9+,4.2,62,6,Colorado,Buena Vista,Davis Face,0,0,-106.1050,38.9174,https://www.mountainproject.com/route/10936475...,1,0,0,0,0,0,0,0,0,0,17,0,D4 is a beautiful route on the right side of t...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1
10565,110794133,25125,Guava Jelly,5.9+,4.1,29,1,Utah,Moab Area,La Sal Mountains,Upper Dakota Crag,Trenchtown,-109.2740,38.4703,https://www.mountainproject.com/route/11079413...,1,0,0,0,0,0,0,0,0,0,17,0,An unbelievable dihedral climb with lots of sm...,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1
13591,114262472,31316,Void of Form,5.9,4.0,47,10,Arizona,Southern Arizona,Mount Lemmon (Santa Catalina Mountains),Pontatoc Canyon,Pontatoc Ridge,-110.8880,32.3454,https://www.mountainproject.com/route/11426247...,1,0,0,0,0,0,0,0,0,0,17,0,Void of Form climbs the long buttress of rock ...,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1
2676,105956634,6358,Camel Toe,5.9+,4.0,38,6,Colorado,Estes Park Valley,Fall River Road (US 34 West),McGregor Slab,0,-105.5850,40.4085,https://www.mountainproject.com/route/10595663...,1,1,0,0,0,0,0,0,0,0,17,0,This is a writeup is based upon firsthand expe...,1,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2999,106028627,7289,Rock Du Jours Direct,5.9+,4.0,179,2,New Hampshire,*Rumney,Main Cliff,0,0,-71.8359,43.8025,https://www.mountainproject.com/route/10602862...,1,0,0,0,0,0,0,0,0,0,17,0,This is perhaps one of the best 5.9s at Rumney...,1,1,0,1,1,1,1,0,0,1,0,1,1,0,0,0,0,1,1,0,0,0,1
429,105737426,1097,Thin Man,5.9 R,4.0,29,1,California,Central Coast,San Luis Obispo,Bishop Peak,Shadow Rock,-120.6960,35.3037,https://www.mountainproject.com/route/10573742...,1,0,0,0,0,0,0,0,0,2,17,0,The last route to the left of Shadow before yo...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1


In [ ]:
# rec=get_recommendations(climb_id, chi2_kernel)
# rec

## NLP (work in progress)

### Import NLP Data (redundent delete later)

In [ ]:
df_nlp = query_helper.query_to_df('SELECT * FROM route_description;')
df_nlp.set_index('id', inplace=True)
df_nlp.head()

In [ ]:
key_words = ['slab', 'traverse', 'roof', 'corner', 'ledge', 'crack', 'face','flake', 'finger', 'fingers',
             'hand', 'hands', 'arch', 'balancy', 'balance', 'jug', 'squeeze', 'mantel', 'sustained',  
             'ramp', 'overhung', 'dihedral', 'sporty', 'heady', 'pump', 'pumpy', 'technical',
             'run out', 'mental', 'well protected', 'chimney', 'offwidth', 'stem', 'arete', 'exposed', 'exposure',
             'crimp','crimpy', 'vertical', 'slabby', 'cave', 'steep', 'bouldery'. 'powerful']

In [ ]:
col_key_words = ['slab', 'traverse', 'roof', 'corner', 'ledge', 'crack', 'face','flake', 'finger',
             'hand', 'arch', 'balancy', 'jug', 'squeeze', 'mantel', 'exposed', 
             'ramp', 'overhung', 'dihedral', 'sporty', 'sustained','pump', 'technical',
             'run out', 'mental', 'well protected', 'chimney', 'offwidth', 'stem', 'arete',
             'crimp', 'vertical', 'cave', 'steep', 'bouldery', 'powerful']

In [ ]:
df_numeric

In [ ]:
df_full = df_numeric.join(df_nlp)
df_full.head()

In [ ]:
df_numeric.head()

### Break descriptions into rope and boulder

#### Rope

In [ ]:
df_sub = df_numeric[(df_numeric['Boulder']==0)]
df_sub = df_sub[(df_numeric['Ice']==0)]
df_sub = df_sub[(df_numeric['Snow']==0)]

df_sub.head()

In [ ]:
df_sub.Boulder.value_counts()

In [ ]:
len(df_sub.index)

In [ ]:
len(df_nlp.loc[df_sub.index])

In [ ]:
nlp_rope =df_nlp.loc[df_sub.index]

In [ ]:
nlp_rope.head()

#### Boulder

#### Tokenizer

In [154]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

In [ ]:
extra_stop = ['climb','climbing', 'crux', 'rope', 'leave', 'use', 'start', 'end',
              'look', 'rock', 'tree', 'follow', 'continue', 'belay', 'photo', 'add', 'climber', 'route', 
              'lot', 'anchor', '...', '1', '2', '3', '4', '--', 'pitch', 'page', 'cold', 'hot', 'warm',
              'belayer', 'fun', 'like', 'unknown', 'rap', 'left', 'right', 'wide', 'leader']


In [ ]:
for word in extra_stop:
    stop_words.add(word)

In [ ]:
stop_words

In [155]:
# Creating our tokenizer function
def spacy_tokenizer(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(text)
    
#     mytokens = [word for word in mytokens if word.pos_ != "PROPN"]
    
    mytokens = [ word if word.pos_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

### TF-IDF Vectorizer

#### Run the vectorizer

In [166]:
df_nlp.loc[105714722,:]

infos    The Central Yellow Wall is a classic must-do p...
Name: 105714722, dtype: object

In [169]:
df_nlp[df_nlp.isna().values]

,infos
id,
117568267,NaN


In [157]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(tokenizer=spacy_tokenizer, min_df=5, max_df=.7)


#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_nlp['infos'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

ValueError: np.nan is an invalid document, expected byte or unicode string.

#### Save it to load later

In [ ]:
# Save the trained model as a pickle string. 
from sklearn.externals import joblib 
import pickle
tfidf_5_7 = pickle.dumps(tfidf_matrix) 

  
# Save the model as a pickle in a file 
joblib.dump(tfidf_matrix, 'tfidf_5_7.pkl') 
  
# Load the model from the file 
# tfidf_loaded = joblib.load('tfidf_5_7.pkl')  
  

### Get original index

In [ ]:
#get index to iterate over
search_range = df_in_range.orig_index

# get index for target in reference to whole matrix
nlp_target_index = int(df_in_range.orig_index[df_in_range['id']==target_id].values)
print('target_index:',nlp_target_index)

In [ ]:
df_nlp.loc[105793305]

In [ ]:
######NEED TO COMBINE WITH get_recommendations so we can get average weighted score#########################
def get_recommendations_for_nlp(idx, kernel_type):

    #value to store scores and indicies
    score_matrix = np.ndarray(shape=(len(df_in_range),2), dtype=float)
    
    for iter_,i in enumerate(search_range.values):
        score = kernel_type(tfidf_matrix[idx],tfidf_matrix[i])
        score_matrix[iter_][0] =  i        ##the index comparison corresponding to the score
        score_matrix[iter_][1] = score     ##the score for the current index

    # Sort the climbs based on the similarity scores
    score_matrix = sorted(score_matrix, key=lambda x: x[1], reverse=True)
    
    # score_matrix
#########################  ADD/calculate SIMilarity VALUE   ###########################

    # # Get the scores of the 20 most similar climbs
    score_matrix = score_matrix[1:20]

    # # Get the climb indices (& cast to ints)
    climb_indices = [int(i[0]) for i in score_matrix]
    
    # Return the top 20 most similar climbs
    return df_numeric.iloc[climb_indices,:]

In [ ]:
rec = get_recommendations_for_nlp(nlp_target_index, cosine_similarity)
rec

In [ ]:
df_nlp.loc[rec.index]

### LDA

#### CV

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.90, min_df=5, stop_words='english', ngram_range=(1,1))

In [ ]:
dtm = cv.fit_transform(nlp_rope['info'].sample(n=1000, random_state=1))

#### LDA Model

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=10,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=32,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

In [ ]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


#### Diagnose model performance with perplexity and log-likelihood
A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good. Let’s check for our model.



In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

In [ ]:
len(lda_model.components_)

In [ ]:
single_topic = lda_model.components_[0]

In [ ]:
top_word_indices = single_topic.argsort()[-10:]

In [ ]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

#### Top words for all groups

In [ ]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-30:]])
    print('\n')

In [ ]:
extra_stop = ['climb','climbing', 'crux', 'rope', 'leave', 'use', 'start', 'end',
              'look', 'rock', 'tree', 'follow', 'continue', 'belay', 'photo', 'add', 'climber', 'route', 
              'lot', 'anchor', '...', '1', '2', '3', '4', '--', 'pitch', 'page', 'cold', 'hot', 'warm',
              'belayer', 'fun', 'like', 'unknown', 'rap', 'left', 'right']



### Try new reverse tokenizer

In [ ]:
key_words = ['slab', 'traverse', 'roof', 'corner', 'ledge', 'crack', 'face','flake', 'bolt', 'finger', 'fingers'
             'cam', 'camalot', 'hand', 'hands', 'arch', 'balancy', 'jug', 'squeeze', 'mantel', 'sustained', 'nut', 
             'gear', 'ramp', 'overhung', 'balance', 'dihedral', 'sporty', 'heady', 'pump', 'pumpy', 'technical',
             'run out', 'mental', 'well protected', 'chimney', 'offwidth', 'stem', 'arete', 'exposed', 'exposure',
             'crimp','crimpy', 'vertical', 'slabby', 'cave', 'steep', 'sidepull','bouldery']

In [ ]:
# Creating our tokenizer function
def spacy_tokenizer_reverse(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(text)
    
#     mytokens = [word for word in mytokens if word.pos_ != "PROPN"]
    
    mytokens = [ word if word.pos_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    
    #grab only my key words
    my_sub_tokens =[]
    for word in set(mytokens):
        if word in key_words:
            my_sub_tokens.append(word)

    # return preprocessed list of tokens
    return my_sub_tokens

#### CV

In [ ]:
cv = CountVectorizer(tokenizer=spacy_tokenizer_reverse,  stop_words='english', ngram_range=(1,1))

In [ ]:
dtm = cv.fit_transform(nlp_rope['info'].sample(n=1000, random_state=1))

#### LDA Model

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=6,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=32,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

print(lda_model)  # Model attributes

In [ ]:
# This can take awhile, we're dealing with a large amount of documents!

lda_output = lda_model.fit_transform(dtm)


#### Diagnose model performance with perplexity and log-likelihood
A model with higher log-likelihood and lower perplexity (exp(-1. * log-likelihood per word)) is considered to be good. Let’s check for our model.



In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(dtm))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(dtm))

# See model parameters
print(lda_model.get_params())

#### Top words for all groups

In [ ]:
for index,topic in enumerate(lda_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')